In [ ]:
# Scraping 
# https://www.skiresort.info/ski-resorts/usa/

In [188]:
# Import dependencies
import pandas as pd
import requests
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint
import json

In [2]:
request = requests.get("https://www.skiresort.info/ski-resorts/usa/")

In [3]:
prep_soup = bs(request.text, 'html.parser')

In [4]:
panel = prep_soup.find('div', class_= "panel panel-primary")

In [5]:
# pprint(panel)

In [20]:
pages = prep_soup.find('ul', class_= "pagination pull-right")

In [21]:
print(pages)

<ul class="pagination pull-right" id="pagebrowser1"> <li> <a href="https://www.skiresort.info/ski-resorts/usa/">«</a> </li> <li> <a href="https://www.skiresort.info/ski-resorts/usa/">‹</a> </li> <li class="active"> <a href="https://www.skiresort.info/ski-resorts/usa/">1</a> </li> <li class=""> <a href="https://www.skiresort.info/ski-resorts/usa/page/2/">2</a> </li> <li class=""> <a href="https://www.skiresort.info/ski-resorts/usa/page/3/">3</a> </li> <li class=""> <a href="https://www.skiresort.info/ski-resorts/usa/page/4/">4</a> </li> <li> <a href="https://www.skiresort.info/ski-resorts/usa/page/2/">›</a> </li> <li> <a href="https://www.skiresort.info/ski-resorts/usa/page/4/">»</a> </li> </ul>


In [7]:
links_with_text = []
for a in pages.find_all('a', href=True): 
    if a.text: 
        links_with_text.append(a['href'])

In [8]:
links_with_text

['https://www.skiresort.info/ski-resorts/usa/',
 'https://www.skiresort.info/ski-resorts/usa/',
 'https://www.skiresort.info/ski-resorts/usa/',
 'https://www.skiresort.info/ski-resorts/usa/page/2/',
 'https://www.skiresort.info/ski-resorts/usa/page/3/',
 'https://www.skiresort.info/ski-resorts/usa/page/4/',
 'https://www.skiresort.info/ski-resorts/usa/page/2/',
 'https://www.skiresort.info/ski-resorts/usa/page/4/']

In [9]:
# name = panel.find_all("div", class_= "panel-body middle-padding").find("div", class_="col-sm-11 col-xs-10")

In [10]:
resort_panels = panel.find_all("div", class_="panel panel-default resort-list-item resort-list-item-image--big")

In [13]:
# resort_titles = []
# for x in resort_panels:
#     title = x.find("a", class_="h3")
# #     print("-------------------------------------------")
#     resort_titles.append(title)
# #     resort_name.append(resort.text)

In [17]:
resort_names = []
resort_links = []
for x in resort_titles:
    resort_names.append(x.text)
    resort_links.append(x["href"])

In [ ]:
# .find_all("a", class_="h3")

In [27]:
resort_names = []
resort_links = []
for x in resort_panels:
    title = x.find("a", class_="h3")
    resort_names.append(title.text)
    resort_links.append(title["href"])
#     print("-------------------------------------------")
#     resort_titles.append(title)

AttributeError: 'list' object has no attribute 'head'

In [52]:
# GETTING ALL NAMES AND LINKS FOR RESORTS
url = 'https://www.skiresort.info/ski-resorts/usa/'

resort_names = []
resort_links = []

for x in range(1, 5):
    request1 = requests.get(url)

    soup1 = bs(request1.text, 'html.parser')
    
    panel = soup1.find('div', class_= "panel panel-primary")
    resort_panels = panel.find_all("div", class_="panel panel-default resort-list-item resort-list-item-image--big")

    for resort in resort_panels:
        title = resort.find("a", class_="h3")
        resort_names.append(title.text)
        resort_links.append(title["href"])
        
    url = f'https://www.skiresort.info/ski-resorts/usa/page/{x+1}'


In [63]:
len(resort_names), len(resort_links)

(523, 523)

In [5]:
test_link = ['https://www.skiresort.info/ski-resort/vail/']
test_links = ['https://www.skiresort.info/ski-resort/vail/', 'https://www.skiresort.info/ski-resort/telluride/',
 'https://www.skiresort.info/ski-resort/beaver-creek/',
 'https://www.skiresort.info/ski-resort/mammoth-mountain/']

In [19]:
# GOING THROUGH RESORT PAGES AND GATHERING RESORT SLOPE LENGTHS
resort_slopes=[]
for x in resort_links:

    request1 = requests.get(x)
    soup1 = bs(request1.text, 'html.parser')
    
    if (soup1.find("a", class_="detail-links link-img shaded zero-pad-bottom chart")):
        
        stuff1 = soup1.find("a", class_="detail-links link-img shaded zero-pad-bottom chart").find("div", class_="description")
        tot_slope= stuff1.find("div").text
        slope_type_lengths = stuff1.find("div", class_="table-graph-first").find_all("td", class_="distance")
    
        splitter = tot_slope.split()
    
        splits = []
        for y in slope_type_lengths:
            split = float(y.text.split()[0])
            splits.append(split)
    
        slope_lengths={'total': float(splitter[1]), 'easy': splits[0], 'intermediate': splits[1], 'difficult': splits[2]}
        resort_slopes.append(slope_lengths)
    else:
        resort_slopes.append('N/A')


[{'total': 234.0, 'easy': 57.0, 'intermediate': 84.0, 'difficult': 93.0},
 {'total': 88.2, 'easy': 6.7, 'intermediate': 51.7, 'difficult': 29.8},
 {'total': 150.0, 'easy': 28.5, 'intermediate': 64.5, 'difficult': 57.0},
 {'total': 89.8, 'easy': 18.4, 'intermediate': 48.5, 'difficult': 22.9},
 {'total': 237.0, 'easy': 12.0, 'intermediate': 114.0, 'difficult': 111.0},
 {'total': 153.0, 'easy': 28.0, 'intermediate': 60.0, 'difficult': 65.0},
 {'total': 135.0, 'easy': 25.0, 'intermediate': 65.0, 'difficult': 45.0},
 {'total': 84.8, 'easy': 25.5, 'intermediate': 30.5, 'difficult': 28.8},
 {'total': 117.0, 'easy': 19.9, 'intermediate': 46.7, 'difficult': 50.2},
 {'total': 94.0, 'easy': 18.0, 'intermediate': 42.0, 'difficult': 34.0},
 {'total': 143.0, 'easy': 11.0, 'intermediate': 53.0, 'difficult': 79.0},
 {'total': 70.0, 'easy': 20.0, 'intermediate': 42.0, 'difficult': 8.0},
 {'total': 100.0, 'easy': 25.0, 'intermediate': 45.0, 'difficult': 30.0},
 {'total': 165.0, 'easy': 25.0, 'intermedia

In [16]:
len(resort_slopes)

525

In [ ]:
resort_slopes

In [6]:
# GOING THROUGH RESORT PAGES AND GATHERING RESORT PRICES
resort_prices = []
for x in resort_links:

    request1 = requests.get(x)

    soup1 = bs(request1.text, 'html.parser')
#     stuff = soup1.find_all("div", class_="detail-links")
#     price = stuff[2].find("td").text

    if (soup1.find(id="selTicketA")):
        
        price = soup1.find(id="selTicketA").text

#     print(price)
#     if price.text: 
#     resort_prices.append(price.text)
        split1 = price.split()
        split2 = float(split1[1].split(',')[0])

        resort_prices.append(split2)
    else:
        resort_prices.append('N/A')


[219.0,
 169.0,
 218.0,
 209.0,
 199.0,
 199.0,
 176.0,
 119.0,
 165.0,
 164.0,
 164.0,
 149.0,
 209.0,
 199.0,
 133.0,
 194.0,
 199.0,
 189.0,
 209.0,
 125.0,
 199.0,
 182.0,
 'N/A',
 151.0,
 99.0,
 115.0,
 170.0,
 169.0,
 'N/A',
 165.0,
 165.0,
 95.0,
 158.0,
 129.0,
 125.0,
 74.0,
 128.0,
 145.0,
 199.0,
 125.0,
 85.0,
 79.0,
 80.0,
 119.0,
 89.0,
 119.0,
 97.0,
 119.0,
 68.58,
 84.0,
 125.0,
 89.0,
 59.0,
 99.0,
 104.0,
 89.0,
 110.0,
 84.0,
 90.0,
 63.0,
 94.0,
 64.0,
 119.0,
 115.0,
 149.0,
 62.0,
 104.0,
 135.0,
 74.0,
 89.0,
 91.0,
 100.0,
 98.0,
 99.0,
 59.0,
 110.0,
 85.0,
 84.0,
 79.0,
 100.0,
 129.0,
 89.0,
 79.0,
 46.0,
 69.0,
 85.0,
 49.0,
 79.0,
 92.0,
 104.0,
 92.0,
 129.0,
 48.0,
 89.0,
 94.0,
 78.0,
 99.0,
 85.0,
 62.0,
 70.0,
 'N/A',
 84.0,
 119.0,
 58.0,
 79.0,
 94.0,
 89.0,
 80.0,
 50.0,
 70.0,
 84.0,
 96.0,
 60.0,
 'N/A',
 49.0,
 50.0,
 79.99,
 95.0,
 'N/A',
 89.0,
 95.0,
 62.0,
 79.0,
 55.0,
 72.0,
 71.0,
 98.0,
 84.0,
 82.0,
 74.0,
 109.0,
 'N/A',
 75.0,
 49.0,


In [ ]:
resort_prices

In [7]:
len(resort_prices)

525

In [255]:
resort_prices = []
# for x in range(0,50):
    
    request1 = requests.get(resort_links[x])

    soup1 = bs(request1.text, 'html.parser')
#     stuff = soup1.find_all("div", class_="detail-links")
#     price = stuff[2].find("td")
    price = soup1.find(id="selTicketA")
#     print(price)
#     if price.text: 
    resort_prices.append(price.text)
#     split1 = price.split()
#     split2 = float(split1[1].split(',')[0])

#     resort_prices.append(split2)
resort_prices

AttributeError: 'NoneType' object has no attribute 'text'

In [8]:
# ALL TOGETHER

# NAMES AND LINKS
# url = 'https://www.skiresort.info/ski-resorts/usa/'

# resort_names = []
# resort_links = []

# for x in range(1, 5):
#     request1 = requests.get(url)

#     soup1 = bs(request1.text, 'html.parser')
    
#     panel = soup1.find('div', class_= "panel panel-primary")
#     resort_panels = panel.find_all("div", class_="panel panel-default resort-list-item resort-list-item-image--big")

#     for resort in resort_panels:
#         title = resort.find("a", class_="h3")
#         resort_names.append(title.text)
#         resort_links.append(title["href"])
        
#     url = f'https://www.skiresort.info/ski-resorts/usa/page/{x+1}'

# GOING THROUGH RESORT PAGES AND GATHERING RESORT SLOPE LENGTHS AND PRICES
all_resort_info=[]


for x in range(0, len(test_links)):
# LENGTHS
    resort_info = {'name': resort_names[x], 'link': resort_links[x]}
    
    
    request1 = requests.get(test_links[x])
    soup1 = bs(request1.text, 'html.parser')
    
    stuff = soup1.find("a", class_="detail-links link-img shaded zero-pad-bottom chart").find("div", class_="description")
    tot_slope= stuff.find("div").text
    slope_type_lengths = stuff.find("div", class_="table-graph-first").find_all("td", class_="distance")
    
    splitter = tot_slope.split()
    
    splits = []
    for y in slope_type_lengths:
        split = float(y.text.split()[0])
        splits.append(split)
    
    slope_lengths={'total': float(splitter[1]), 'easy': splits[0], 'intermediate': splits[1], 'difficult': splits[2]}

    resort_info["slopes"] = slope_lengths    
    
# PRICES

    if (soup1.find(id="selTicketA")):
        
        price = soup1.find(id="selTicketA").text
        
        split1 = price.split()
        split2 = float(split1[1].split(',')[0])

        resort_info["price"]= split2
        
    else:
        
        resort_info["price"]='N/A'

    all_resort_info.append(resort_info)

In [9]:
all_resort_info

[{'name': ' Vail ',
  'link': 'https://www.skiresort.info/ski-resort/vail/',
  'slopes': {'total': 234.0,
   'easy': 57.0,
   'intermediate': 84.0,
   'difficult': 93.0},
  'price': 219.0},
 {'name': ' Telluride ',
  'link': 'https://www.skiresort.info/ski-resort/telluride/',
  'slopes': {'total': 88.2,
   'easy': 6.7,
   'intermediate': 51.7,
   'difficult': 29.8},
  'price': 169.0},
 {'name': ' Beaver Creek ',
  'link': 'https://www.skiresort.info/ski-resort/beaver-creek/',
  'slopes': {'total': 150.0,
   'easy': 28.5,
   'intermediate': 64.5,
   'difficult': 57.0},
  'price': 218.0},
 {'name': ' Mammoth Mountain ',
  'link': 'https://www.skiresort.info/ski-resort/mammoth-mountain/',
  'slopes': {'total': 89.8,
   'easy': 18.4,
   'intermediate': 48.5,
   'difficult': 22.9},
  'price': 209.0}]

In [128]:
test = ['https://www.skiresort.info/ski-resort/vail/', 'https://www.skiresort.info/ski-resort/telluride/', 'https://www.skiresort.info/ski-resort/white-pass/']

resort_zips = []
counter = 1
for x in resort_links:
#     counter = i + 1
    url= x + 'lift-operator/'
    try:
        request1 = requests.get(url)
        soup1 = bs(request1.text, 'html.parser')
    #     stuff = soup1.find_all("div", class_="detail-links")
    #     price = stuff[2].find("td").text

    #     if (soup1.find(id="selTicketA")):

        thing = soup1.find("div", class_="panel-body middle-padding")
        lilzip = thing.find("p")
    #     lilthingthang = lilzip.find("br")
        bigthang =  lilzip.findAll('br')

    #     for x in bigthang:
    #         if x.nextSibling:
    #             print(x.nextSibling)

        for x in range(len(bigthang)-2):   
            if bigthang[x].nextSibling[1].isdigit():
                
                zippper = bigthang[x].nextSibling.split()[0]
                if len(zippper) >= 5:
                    resort_zips.append(zippper)

       
    except:
        
        resort_zips.append('N/A')
        
        
    resort_zips.append(f'--{counter}--')
    counter +=1
#     print('****')
#     one = bigthang[1].nextSibling
#     zipppy = []
#     for x in range(1,6):
        
#         zipppy.append(one[x])
#     zippo = "".join(zipppy)
#     print(one)
#     print('&&&&')
#     print(zipppy)
#     print(zippo)
# #     print(one)
#     print('*********')
#     print(lilzip)
#     print('---------------------')
#     print(lilzip.text)
#     print('---------------------')
#     print(lilthingthang)
#     print('---------------------')
# #     print(price)
#     if price.text: 
#     resort_prices.append(price.text)
#         split1 = price.split()
#         split2 = float(split1[1].split(',')[0])

#         resort_prices.append(split2)
#     else:
#         resort_prices.append('N/A')

In [129]:
resort_zips

['81658',
 '--1--',
 '81435',
 '--2--',
 '81658',
 '--3--',
 '93546',
 '--4--',
 '81612',
 '--5--',
 '80424',
 '--6--',
 '80435',
 '--7--',
 '05751',
 '--8--',
 '04261',
 '--9--',
 '96150',
 '--10--',
 '80482',
 '--11--',
 '05672',
 '--12--',
 '96146',
 '--13--',
 '80487',
 '--14--',
 '84060',
 '--15--',
 '59716',
 '--16--',
 '81612',
 '--17--',
 '80443',
 '--18--',
 '96146',
 '--19--',
 '84092',
 '--20--',
 '81612',
 '--21--',
 '83025',
 '--22--',
 '84060',
 '--23--',
 '81224',
 '--24--',
 '84310',
 '--25--',
 '12997',
 '--26--',
 '97702',
 '--27--',
 '96161',
 '--28--',
 '59716',
 '--29--',
 '84092',
 '--30--',
 '83353',
 '--31--',
 '83864',
 '--32--',
 '80435',
 '--33--',
 '97041',
 '--34--',
 '05155',
 '--35--',
 '98801',
 '--36--',
 '87525',
 '--37--',
 '89511',
 '--38--',
 '81612',
 '--39--',
 '95724',
 '--40--',
 '59937',
 '--41--',
 '87710',
 '--42--',
 '81147',
 '--43--',
 '05674',
 '--44--',
 '81301',
 '--45--',
 '04947',
 '--46--',
 '84121',
 '--47--',
 '05149',
 '--48--',
 

In [177]:

resort_zips1 = []
# counter = 1
for x in resort_links:
    lilzipz = []
    url= x + 'lift-operator/'
    try:
        
        request1 = requests.get(url)
        soup1 = bs(request1.text, 'html.parser')

        
        thing = soup1.find("div", class_="panel-body middle-padding")
        lilzip = thing.find("p")
        lilzipz.append(lilzip.text.split()[0])
        bigthang =  lilzip.findAll('br')
        for x in bigthang:
#             counter = 1
            if len(x.nextSibling) > 1:
#                 print(x.nextSibling)
                if x.nextSibling[1].isdigit():
                    zippper = x.nextSibling.split()[0]
#                     print(zippper)
                    lilzipz.append(zippper)

        resort_zips1.append(lilzipz[-1])


       
    except:
        
        resort_zips1.append('N/A')
        
        
#     resort_zips1.append(f'--{counter}--')
#     counter +=1

In [178]:
resort_zips1

['81658',
 '81435',
 '81658',
 '93546',
 '81612',
 '80424',
 '80435',
 '05751',
 '04261',
 '96150',
 '80482',
 '05672',
 '96146',
 '80487',
 '84060',
 '59716',
 '81612',
 '80443',
 '96146',
 '84092',
 '81612',
 '83025',
 '84060',
 '81224',
 '84310',
 '12997',
 '97702',
 '96161',
 '59716',
 '84092',
 '83353',
 '83864',
 '80435',
 '97041',
 '05155',
 '98801',
 '87525',
 '89511',
 '81612',
 '95724',
 '59937',
 '87710',
 '81147',
 '05674',
 '81301',
 '04947',
 '84121',
 '05149',
 '98229',
 '05477',
 '05859',
 '84121',
 '84719',
 '95223',
 '99587',
 '03575',
 '05356',
 '59068',
 '05464-9537',
 '59715',
 '83702',
 '95646',
 '12853',
 '95735',
 '98022',
 '99109',
 '97028',
 '84317',
 '03251',
 '88312-9600',
 '12442',
 '83615',
 '12496',
 '86002',
 '97028',
 '83414',
 '14731-0370',
 '98826',
 '03580',
 '55612',
 '80466',
 '80444',
 '05751',
 '98937',
 '59827',
 '81643',
 '03249',
 '93529',
 '93634',
 '95724',
 '05673',
 '59858',
 '59640',
 '95364',
 '03860',
 '03812',
 '03255',
 '84604',
 '838

In [179]:
len(resort_zips1)

523

In [240]:
len('14731-0279')

10

In [243]:
xz = '14731-0279'
yyy = xz.split('-')
yyy[0]

'14731'

In [244]:
clean_zips = []

for x in resort_zips1:
    if len(x) > 5:
        y= x.split('-')[0]
        clean_zips.append(y)
    else:
        clean_zips.append(x)

In [246]:
len(clean_zips)

523

In [86]:
titty = ['https://www.skiresort.info/ski-resort/white-pass/', 'https://www.skiresort.info/ski-resort/nutt-hill-plymouth/']
tittyzip = []
for x in titty:
    
    url1= x + 'lift-operator/'
    try:
        request1 = requests.get(url1)
        soup1 = bs(request1.text, 'html.parser')
    #     stuff = soup1.find_all("div", class_="detail-links")
    #     price = stuff[2].find("td").text

    #     if (soup1.find(id="selTicketA")):

        thing = soup1.find("div", class_="panel-body middle-padding")
        lilzip = thing.find("p")
    #     lilthingthang = lilzip.find("br")
        bigthang =  lilzip.findAll('br')

    #     for x in bigthang:
    #         if x.nextSibling:
    #             print(x.nextSibling)

        for x in range(len(bigthang)-2):   
            if bigthang[x].nextSibling[1].isdigit():
                
                zippper = bigthang[x].nextSibling.split()[0]
                if len(zippper) >= 5:
                    resort_zips.append(zippper)

       
    except:
        
        tittyzip.append('N/A')
        
        
    tittyzip.append('----')
    


In [87]:
tittyzip

['N/A', '----', 'N/A', '----']

In [171]:
tittyzipz = []
request1 = requests.get('https://www.skiresort.info/ski-resort/vail/lift-operator/')
soup1 = bs(request1.text, 'html.parser')

lilzipz = []

thing = soup1.find("div", class_="panel-body middle-padding")
lilzip = thing.find("p")
lilzipz.append(lilzip.text.split()[0])
bigthang =  lilzip.findAll('br')
for x in bigthang:
    counter = 1
    if len(x.nextSibling) > 1:
        print(x.nextSibling)
        if x.nextSibling[1].isdigit():
            zippper = x.nextSibling.split()[0]
            print(zippper)
            lilzipz.append(zippper)

tittyzipz.append(lilzipz[-1])
lilzipz


#     tittyzipz.append(f'--{counter}--')     
#     print(x.nextSibling)
#     if x.nextSibling[1]:
#         print(x)
# print(bigthang[3].nextSibling[1])
#     if isinstance(bigthang[x].nextSibling,str):
    
#     [1].isdigit():
                
#         zippper = bigthang[x].nextSibling.split()[0]
#         if len(zippper) >= 5:
#         tittyzipz.append(zippper)

 P.O.Box 7
 81658 Vail
81658
 United States
 Tel.: 
 Email: 


['Vail', '81658']

In [170]:
tittyzipz



['84121']

In [184]:
zip_csv = 'us-zip-code-latitude-and-longitude.csv'
zip_df = pd.read_csv(zip_csv)
zip_df.head()

,Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint
48834;Fenwick;MI;43.141649;-85.04948;-5;1;43.141649,-85.04948
55304;Andover;MN;45.254715;-93.28652;-6;1;45.254715,-93.28652
55422;Minneapolis;MN;45.014764;-93.33965;-6;1;45.014764,-93.33965
29079;Lydia;SC;34.296064;-80.11319;-5;1;34.296064,-80.11319
29390;Duncan;SC;34.888237;-81.96902;-5;1;34.888237,-81.96902


In [189]:
thing = 'us-zip-code-latitude-and-longitude.json'
print(json.dumps(thing, indent=4, sort_keys=True))

"us-zip-code-latitude-and-longitude.json"


In [190]:
# Opening JSON file 
f = open('us-zip-code-latitude-and-longitude.json',) 
  
# returns JSON object as  
# a dictionary 
data = json.load(f) 
  
# Iterating through the json 
# list 
# for i in data['emp_details']: 
#     print(i) 
  
# Closing file 
f.close() 

In [195]:
data

[{'datasetid': 'us-zip-code-latitude-and-longitude',
  'recordid': 'a77fe7b3cf2e42c38c064bc3e94df51bde9bb720',
  'fields': {'city': 'Fenwick',
   'zip': '48834',
   'dst': 1,
   'geopoint': [43.141649, -85.04948],
   'longitude': -85.04948,
   'state': 'MI',
   'latitude': 43.141649,
   'timezone': -5},
  'geometry': {'type': 'Point', 'coordinates': [-85.04948, 43.141649]},
  'record_timestamp': '2018-02-09T08:33:38.603-08:00'},
 {'datasetid': 'us-zip-code-latitude-and-longitude',
  'recordid': 'ed12cbacde531be4a3c94f5e839e114b26337309',
  'fields': {'city': 'Andover',
   'zip': '55304',
   'dst': 1,
   'geopoint': [45.254715, -93.28652],
   'longitude': -93.28652,
   'state': 'MN',
   'latitude': 45.254715,
   'timezone': -6},
  'geometry': {'type': 'Point', 'coordinates': [-93.28652, 45.254715]},
  'record_timestamp': '2018-02-09T08:33:38.603-08:00'},
 {'datasetid': 'us-zip-code-latitude-and-longitude',
  'recordid': '4aa0ece0cb0898af5d761bf95f74680901578527',
  'fields': {'city': 'M

In [232]:
for x in data:
    print(x)

{'datasetid': 'us-zip-code-latitude-and-longitude', 'recordid': 'a77fe7b3cf2e42c38c064bc3e94df51bde9bb720', 'fields': {'city': 'Fenwick', 'zip': '48834', 'dst': 1, 'geopoint': [43.141649, -85.04948], 'longitude': -85.04948, 'state': 'MI', 'latitude': 43.141649, 'timezone': -5}, 'geometry': {'type': 'Point', 'coordinates': [-85.04948, 43.141649]}, 'record_timestamp': '2018-02-09T08:33:38.603-08:00'}
{'datasetid': 'us-zip-code-latitude-and-longitude', 'recordid': 'ed12cbacde531be4a3c94f5e839e114b26337309', 'fields': {'city': 'Andover', 'zip': '55304', 'dst': 1, 'geopoint': [45.254715, -93.28652], 'longitude': -93.28652, 'state': 'MN', 'latitude': 45.254715, 'timezone': -6}, 'geometry': {'type': 'Point', 'coordinates': [-93.28652, 45.254715]}, 'record_timestamp': '2018-02-09T08:33:38.603-08:00'}
{'datasetid': 'us-zip-code-latitude-and-longitude', 'recordid': '4aa0ece0cb0898af5d761bf95f74680901578527', 'fields': {'city': 'Minneapolis', 'zip': '55422', 'dst': 1, 'geopoint': [45.014764, -93.

{'datasetid': 'us-zip-code-latitude-and-longitude', 'recordid': '752c90905f35f2d7c9610bc670e555c9e117b1d7', 'fields': {'city': 'Cypress', 'zip': '90630', 'dst': 1, 'geopoint': [33.817481, -118.0399], 'longitude': -118.0399, 'state': 'CA', 'latitude': 33.817481, 'timezone': -8}, 'geometry': {'type': 'Point', 'coordinates': [-118.0399, 33.817481]}, 'record_timestamp': '2018-02-09T08:33:38.603-08:00'}
{'datasetid': 'us-zip-code-latitude-and-longitude', 'recordid': 'a49d5f690ecd905b026cba076496e46c2dd7f8e2', 'fields': {'city': 'Nahcotta', 'zip': '98637', 'dst': 1, 'geopoint': [46.514625, -123.76903], 'longitude': -123.76903, 'state': 'WA', 'latitude': 46.514625, 'timezone': -8}, 'geometry': {'type': 'Point', 'coordinates': [-123.76903, 46.514625]}, 'record_timestamp': '2018-02-09T08:33:38.603-08:00'}
{'datasetid': 'us-zip-code-latitude-and-longitude', 'recordid': '02afe33159c59a051e0341ed9336237d20920687', 'fields': {'city': 'Wapato', 'zip': '98951', 'dst': 1, 'geopoint': [46.457731, -120.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [224]:
data[0]

{'datasetid': 'us-zip-code-latitude-and-longitude',
 'recordid': 'a77fe7b3cf2e42c38c064bc3e94df51bde9bb720',
 'fields': {'city': 'Fenwick',
  'zip': '48834',
  'dst': 1,
  'geopoint': [43.141649, -85.04948],
  'longitude': -85.04948,
  'state': 'MI',
  'latitude': 43.141649,
  'timezone': -5},
 'geometry': {'type': 'Point', 'coordinates': [-85.04948, 43.141649]},
 'record_timestamp': '2018-02-09T08:33:38.603-08:00'}

In [226]:
data[0]['geometry']['coordinates']

[-85.04948, 43.141649]

In [247]:
testzips = ['81658',
 '81435',
 '81658',
 '93546']
lats = []
for x in clean_zips:
    curr_len = len(lats)    
    for y in data:
        if (x ==y['fields']['zip']):
            lats.append(y['geometry']['coordinates'])
        
    if len(lats) == curr_len:
        lats.append('N/A')
lats        


[[-106.497314, 39.624171],
 [-107.88874, 37.932874],
 [-106.497314, 39.624171],
 [-118.86279, 37.621587],
 [-106.882774, 39.223416],
 [-106.03747, 39.478893],
 [-105.97937, 39.607479],
 [-72.79419, 43.659839],
 [-70.87599, 44.570532],
 [-120.00992, 38.908741],
 [-105.78359, 39.915508],
 [-72.70225, 44.475387],
 [-120.21804, 39.180983],
 [-106.87521, 40.502772],
 [-111.49362, 40.648541],
 [-111.32651, 45.268367],
 [-106.882774, 39.223416],
 [-106.14029, 39.532506],
 [-120.21804, 39.180983],
 [-111.80902, 40.558748],
 [-106.882774, 39.223416],
 [-110.694002, 43.443851],
 [-111.49362, 40.648541],
 [-106.96556, 38.865307],
 [-111.81304, 41.364638],
 [-73.815204, 44.145678],
 [-121.30891, 44.020483],
 [-120.22805, 39.339574],
 [-111.32651, 45.268367],
 [-111.80902, 40.558748],
 [-114.18191, 43.661373],
 [-116.55101, 48.362033],
 [-105.97937, 39.607479],
 [-121.59252, 45.503224],
 [-72.84497, 43.158484],
 [-120.32881, 47.425497],
 [-105.50602, 36.575944],
 [-119.7606, 39.403178],
 [-106.8827

In [248]:
len(lats)

523

In [ ]:
for br in soup.findAll('br'):
    next_s = br.nextSibling
    if not (next_s and isinstance(next_s,NavigableString)):
        continue
    next2_s = next_s.nextSibling
    if next2_s and isinstance(next2_s,Tag) and next2_s.name == 'br':
        text = str(next_s).strip()
        if text:
            print "Found:", next_s